In [2]:
import requests
import pandas as pd

In [34]:
US = requests.get('https://api.census.gov/data/2017/abscs?get=NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP&for=us:*&NAICS2017=00&key=a4bcb4614372e8f229066cbce694ccdb6b558ef4')

In [35]:
state = requests.get('https://api.census.gov/data/2017/abscs?get=NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP&for=state:*&NAICS2017=00&key=a4bcb4614372e8f229066cbce694ccdb6b558ef4')

In [24]:
countyinstate = requests.get('https://api.census.gov/data/2017/abscs?get=NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP&for=county:*&in=state:*&NAICS2017=00&key=a4bcb4614372e8f229066cbce694ccdb6b558ef4')

In [36]:
data = US.json()

In [37]:
data2=state.json()

In [38]:
data3 = countyinstate.json()

In [40]:
companysummaryUS = pd.DataFrame (data[1:], columns= data[0])

In [41]:
companysummaryUS

,NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP,NAICS2017,us
0,United States,0100000US,Total for all sectors,004,001,00,001,859735,00,1
1,United States,0100000US,Total for all sectors,001,020,00,001,322076,00,1
2,United States,0100000US,Total for all sectors,002,020,00,001,78091,00,1
3,United States,0100000US,Total for all sectors,003,020,00,001,206437,00,1
4,United States,0100000US,Total for all sectors,001,001,00,001,5744643,00,1
...,...,...,...,...,...,...,...,...,...,...
358,United States,0100000US,Total for all sectors,001,001,63,004,24082,00,1
359,United States,0100000US,Total for all sectors,001,022,00,003,0,00,1
360,United States,0100000US,Total for all sectors,001,001,92,003,136094,00,1
361,United States,0100000US,Total for all sectors,001,001,64,004,23096,00,1


In [28]:
countysummary = pd.DataFrame (data3[1:], columns= data3[0])

In [29]:
countysummary

,NAME,GEO_ID,NAICS2017_LABEL,SEX,ETH_GROUP,RACE_GROUP,VET_GROUP,FIRMPDEMP,NAICS2017,state,county
0,"Warren County, Tennessee",0500000US47177,Total for all sectors,001,001,00,001,561,00,47,177
1,"Warren County, Tennessee",0500000US47177,Total for all sectors,002,001,00,001,0,00,47,177
2,"Warren County, Tennessee",0500000US47177,Total for all sectors,003,001,00,001,296,00,47,177
3,"Warren County, Tennessee",0500000US47177,Total for all sectors,004,001,00,001,0,00,47,177
4,"Warren County, Tennessee",0500000US47177,Total for all sectors,001,020,00,001,0,00,47,177
...,...,...,...,...,...,...,...,...,...,...,...
51725,"Olmsted County, Minnesota",0500000US27109,Total for all sectors,001,001,00,002,200,00,27,109
51726,"Olmsted County, Minnesota",0500000US27109,Total for all sectors,001,001,00,003,0,00,27,109
51727,"Olmsted County, Minnesota",0500000US27109,Total for all sectors,001,001,00,004,2084,00,27,109
51728,"Olmsted County, Minnesota",0500000US27109,Total for all sectors,096,096,96,096,2327,00,27,109
